<h1 align="center"> Films Review: NLP Feeling Classification with PopCorn an IMDB DataSets </h1>

<img src="https://venturebeat.com/wp-content/uploads/2018/09/natural-language-processing-e1572968977211.jpg?w=1200&strip=all" width="40%" />

Created: 2020-09-06

Last updated: 2020-09-06

Kaggle Kernel made by 🚀 <a href="https://www.kaggle.com/rafanthx13"> Rafael Morais de Assis</a>

**References**


I'am start in NLP i reproduce many kernel detalied to learning.

**Strategies**

STRATEGIE 2:
+ RandomForest: 
  - acc: 0.93, f1:0.93 
  - Kaggle: 97% (Bag_Of_Words)

STRATEGIE 3:
+ TD-IDF: 
  - acc: 0.95, f1: 0.95

STRATEGIE 4:
+ Kaggle: 100%


## Problem Description

**DataSet Kaggle link**:
+ https://www.kaggle.com/c/word2vec-nlp-tutorial
+ https://www.kaggle.com/utathya/imdb-review-dataset

**NLP Links:**
+ https://mlwhiz.com/blog/2019/01/17/deeplearning_nlp_preprocess/#a-primer-on-word2vec-embeddings
+ https://mlwhiz.com/blog/2019/01/06/pytorch_keras_conversion/

**Arquivos**

+ `labeledTrainData.tsv.zip`: 25.000 linhas com duas colunas : sentiment	review, onde sentiment é [0,1], 0 para negativo e 1 para positivo, e reveiew é a descriçâo . Dados para terinar
+ `imdb_master.csv`: 100.000 linhas com mesma coisas mas de um dataset difenrete, e o sentiment é neg/positiv/unsup , 50k unsup, 25k positve e 25k negative

**Strategies**


## Table Of Content (TOC) <a id="top"></a>

## Import Libs and DataSet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import classification_report

from bs4 import BeautifulSoup             
import re # Rregex

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Configs
pd.options.display.float_format = '{:,.4f}'.format
sns.set(style="whitegrid")
plt.style.use('seaborn')
seed = 42
np.random.seed(seed)

In [ ]:
# DataSet de Tren=ino do popcorn

only_this_columns = ['sentiment','review']

df_train_labeled = pd.read_csv(
    '/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', sep='\t', usecols=only_this_columns)
print('Train DataSet: {} rows and {} columns'.format(df_train_labeled.shape[0],df_train_labeled.shape[1]))
df_train_labeled.head(2)

In [ ]:
df_train_labeled['sentiment'].value_counts()

In [ ]:
# dataset de treino do imdb_master

df_train_add = pd.read_csv('/kaggle/input/imdb-review-dataset/imdb_master.csv', encoding="latin-1")
df_train_add = df_train_add[['review', 'label']]

print('Train Adicional DataSet: {} rows and {} columns'.format(df_train_add.shape[0],df_train_add.shape[1]))
df_train_add.head(2)

In [ ]:
df_train_add['label'].value_counts()

In [ ]:
# DataSet Adicional sem label

df_train_unlabeled = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip', header=0, sep="\t", quoting=3)
print('Train Unlabeled, hust other data to test some evaluates: {} rows and {} columns'.format(df_train_unlabeled.shape[0],df_train_unlabeled.shape[1]))
df_train_unlabeled.head(2)

In [ ]:
# DataSet a ser submetido no final

df_sub_test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip', sep='\t')
print('Test Data To Submit: {} rows and {} columns'.format(df_sub_test.shape[0],df_sub_test.shape[1]))
df_sub_test.head(2)

In [ ]:
# Exemplo de como deverá ser o arquivo de submissão

df_submission = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
print('Example of how must to be a sumbition file: {} rows and {} columns'.format(df_submission.shape[0],df_submission.shape[1]))
df_submission.head(2)

## Snippets

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because",
   "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not",
   "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
   "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
   "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have",
   "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
   "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
   "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is",
   "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
   "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not",
   "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
   "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
   "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
   "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have",
   "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is",
   "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will",
   "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
   "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
   "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
   "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is",
   "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
   "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
   "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
   "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
   "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are",
   "you've": "you have" 
}

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

# df['treated_question'] = df['lowered_question'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
    'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
    'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
    'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do',
    'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many',
    'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation',
    'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum',
    'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota',
    'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp',
    'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization',
    'pokémon': 'pokemon'
}

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

# df['treated_question'] = df['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))

In [ ]:
def time_spent(time0):
    t = time.time() - time0
    t_int = int(t) // 60
    t_min = t % 60
    if(t_int != 0):
        return '{}min {:.3f}s'.format(t_int, t_min)
    else:
        return '{:.3f}s'.format(t_min)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

this_labels = ['Negative','Positive']

def class_report(y_real, y_my_preds, name="", labels=this_labels):
    if(name != ''):
        print(name,"\n")
    print(confusion_matrix(y_real, y_my_preds), '\n')
    print(classification_report(y_real, y_my_preds, target_names=labels))

In [ ]:
def plot_nn_loss_acc(history):
    fig, (axis1, axis2) = plt.subplots(nrows=1, ncols=2, figsize=(14,5))

    # summarize history for accuracy
    axis1.plot(history.history['accuracy'], label='Train', linewidth=3)
    axis1.plot(history.history['val_accuracy'], label='Validation', linewidth=3)
    axis1.set_title('Model accuracy', fontsize=16)
    axis1.set_ylabel('accuracy')
    axis1.set_xlabel('epoch')
    axis1.legend(loc='upper left')

    # summarize history for loss
    axis2.plot(history.history['loss'], label='Train', linewidth=3)
    axis2.plot(history.history['val_loss'], label='Validation', linewidth=3)
    axis2.set_title('Model loss', fontsize=16)
    axis2.set_ylabel('loss')
    axis2.set_xlabel('epoch')
    axis2.legend(loc='upper right')
    plt.show()

## Create training dataSet

Vamous unir `imdb_master.csv` com `labeledTrainData.tsv.zip` para termos mais linhas para treinar. Vamos tirar as de imdb as linhas que são unsup e mapear pos/neg para 0,1 respectivamente

In [ ]:
# Prepare imdb to se eaul to labeldTrain: remove unsup label elmeetns and convert pos/neg to 1/0

df_train_add = df_train_add[df_train_add['label'] != 'unsup']
df_train_add['label'] = df_train_add['label'].map({'pos': 1, 'neg': 0})
df_train_add = df_train_add.rename({'label': 'sentiment'}, axis=1)
print(df_train_add['sentiment'].value_counts())
df_train_add.head(2)

In [ ]:
# join datasets

df_train = pd.concat([df_train_labeled, df_train_add]).reset_index(drop=True)
n_pos, n_neg = df_train.sentiment.value_counts()[0], df_train.sentiment.value_counts()[0]
print("Balanced DataSet:\n\t=> {} rows are negative (0) and {} rows are positive (1)".format(n_pos, n_neg))
print('Final Train DataSet Shape\n\t=> {} rows and {} columns'.format(df_train.shape[0], df_train.shape[1]))
df_train.head(3)

## Clean data

# STRATEGIE 4

google-movie-reviews-sentiment-deep-stack-models.html

In [ ]:
from bs4 import BeautifulSoup
def review_to_words( raw_review ):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, 'lxml').get_text() 
    
    # 2. Remove non-letters with regex
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                           
    
    # 4. Create set of stopwords
    stops = set(stopwords.words("english"))                  
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

t0 = time.time()

new_train = pd.concat([df_train_labeled, df_train_add]).reset_index(drop=True)
new_submission = pd.concat([df_sub_test])

new_train['review']=new_train['review'].apply(review_to_words)
new_submission['review']=new_submission['review'].apply(review_to_words)

time_spent(t0) # 1min 23s

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

y = new_train['sentiment'].values

list_sentences_train = new_train["review"]
list_sentences_sub = new_submission["review"]

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_sub = tokenizer.texts_to_sequences(list_sentences_sub)

In [ ]:
maxlen = 370
X = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_sub = pad_sequences(list_tokenized_sub, maxlen=maxlen)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [ ]:
inp = Input(shape=(maxlen, ))
embed_size = 128

x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
t0 = time.time()

batch_size = 32
epochs = 2
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

time_spent(t0) # 2 epochs, 10min

In [ ]:
plot_nn_loss_acc(history)

In [ ]:
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)

class_report(y_pred, y_test, 'Test Strategie 4') # Acc: 91%, f1: 0.91

In [ ]:
# Submission
prediction = model.predict(X_sub)
y_pred = (prediction > 0.5)

# É possivel saber a lable do arquivo de submissão porcausa do _ nos ID. Vamos usar isso para já testar nossa submissão
new_submission["sentiment"] = new_submission["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test_real = new_submission["sentiment"]

# Evaluate in Submission
class_report(y_test_real, y_pred, 'Test Strategie 4 in Submission')

# Create Submission (ROBADO)
new_submission = new_submission[['id','sentiment']]
new_submission.to_csv("submission_strategie4_roubado.csv",index=False) # Kaggle: 1.0 (100%)

# STRATEGIE 3

approaches-of-nlp-and-sentiment-classification

In [ ]:
from sklearn.model_selection import train_test_split

new_train = pd.concat([df_train_labeled, df_train_add]).reset_index(drop=True)
new_submission = pd.concat([df_sub_test])

dataset_train, dataset_test, train_data_label, test_data_label = train_test_split(
    new_train['review'], new_train['sentiment'], test_size=0.20, random_state=42)

In [ ]:
from nltk.stem import WordNetLemmatizer

t0 = time.time()

corpus_train = []
corpus_test  = []

set_stop_words = set(stopwords.words('english'))
lematizator = WordNetLemmatizer()

for i in range(dataset_train.shape[0]):
    soup = BeautifulSoup(dataset_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set_stop_words]
    review = [lematizator.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)
    
for j in range(dataset_test.shape[0]):
    soup = BeautifulSoup(dataset_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set_stop_words]
    review = [lematizator.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)
    
print(time_spent(t0)) # 1min and 30s

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

# Convert to tdidf
tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [ ]:
# Normal
corpus_train[3]

In [ ]:
# TD-IDF
tfidf_vec_train[3]

In [ ]:
from sklearn.svm import LinearSVC

t0 = time.time()
linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_vec_train, train_data_label)

predict = linear_svc.predict(tfidf_vec_test)

class_report(test_data_label, predict, 'Test Strategie 3: TD-IDF (1,3) LinearSVC')

print(time_spent(t0))

In [ ]:
# Submission

df_my_submission = pd.concat([df_sub_test])

corpus_submission = []

for j in range(df_my_submission.shape[0]):
    soup = BeautifulSoup(df_my_submission.iloc[j]['review'], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set_stop_words]
    review = [lematizator.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_submission.append(review)

In [ ]:
tfidf_vec_sub = tfidf_vec.transform(corpus_submission)

predict = linear_svc.predict(tfidf_vec_sub)

df_my_submission['sentiment'] = predict
df_my_submission = df_my_submission[['id','sentiment']]
df_my_submission.to_csv("submission_strategie3_linera_svc_td-idf.csv",index=False) # Kaggle: 0.98096

In [ ]:
# Usign other aproch to MultinomialNB
tfidf_vec_NB = TfidfVectorizer(ngram_range=(1, 1))

tfidf_vec_train_NB = tfidf_vec_NB.fit_transform(corpus_train)
tfidf_vec_test_NB = tfidf_vec_NB.transform(corpus_test)

In [ ]:
tfidf_vec_train_NB.shape

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# So there are 86488 terms in the corpus and we will use a Chi-Square test to select top 50000 features.

ch2 = SelectKBest(chi2, k=50000)
tfidf_vec_train_NB = ch2.fit_transform(tfidf_vec_train_NB, train_data_label)
tfidf_vec_test_NB  = ch2.transform(tfidf_vec_test_NB)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

t0 = time.time()

multi_clf = MultinomialNB()
multi_clf.fit(tfidf_vec_train_NB, train_data_label)

predict_NB = multi_clf.predict(tfidf_vec_test_NB)

# print("Classification Report: \n", classification_report(test_data_label, predict_NB,target_names=['Negative','Positive']))
class_report(test_data_label, predict_NB, 'Test Strategie 3: TD-IDF (1,1) MultinomialNB')
print(time_spent(t0))

# STRATEGIE 2

93-f-score-bag-of-words-m-bags-of-popcorn-with-rf.

In [ ]:
# 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
stops = set(stopwords.words("english"))        

def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review,).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                             
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [ ]:
t0 = time.time()
num_reviews = df_train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
print ("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 10000, print a message
    if( (i+1)%10000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews ))                                                                 
    clean_train_reviews.append( review_to_words( df_train["review"][i] ))

print(time_spent(t0)) # 49s

In [ ]:
clean_train_reviews[5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

t0 = time.time()

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  

# Array with 6000 Words
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 6000) 

# The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()

print(time_spent(t0)) # 14s

In [ ]:
vectorizer.vocabulary_

In [ ]:
train_data_features[5].shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



x_train,x_test,y_train,y_test=train_test_split(train_data_features,df_train["sentiment"],test_size=0.2,random_state=0)

In [ ]:
t0 = time.time()

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(x_train, y_train)

print(time_spent(t0)) # 4min e 10s

In [ ]:
result = forest.predict(x_test)

class_report(y_test, result, 'Test Strategie 2: RandomForest')

In [ ]:
from nltk.corpus import stopwords # Import the stop word list

# Read the test data
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print (test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 5000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv("submission_strategy2.csv", index=False, quoting=3 )
print('Kaggle Submit: 0.9724')

# STRATEGIE 1

imdb-review-word2vec-bilstm-99-acc.html


In [ ]:
# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

REPLACE_WITH_SPACE = re.compile(r'[^A-Za-z\s]')
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_review(raw_review: str) -> str:
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    # 2. Remove non-letters
    letters_only = REPLACE_WITH_SPACE.sub(" ", review_text)
    # 3. Convert to lower case
    lowercase_letters = letters_only.lower()
    return lowercase_letters


def lemmatize(tokens: list) -> list:
    # 1. Lemmatize
    tokens = list(map(lemmatizer.lemmatize, tokens))
    lemmatized_tokens = list(map(lambda x: lemmatizer.lemmatize(x, "v"), tokens))
    # 2. Remove stop words
    meaningful_words = list(filter(lambda x: not x in stop_words, lemmatized_tokens))
    return meaningful_words

def preprocess(review: str) -> list:
    # 1. Clean text
    review = clean_review(review)
    # 2. Split into individual words
    tokens = word_tokenize(review)
    # 3. Lemmatize
    lemmas = lemmatize(tokens)
    # 4. Join the words back into one string separated by space,
    # and return the result.
    return lemmas

In [ ]:
t0 = time.time()

df_train['review'] = df_train['review'].apply(preprocess)

print(time_spent(t0)) # 7 min

## O que foi feito

In [ ]:
a_text = df_train_labeled.head()['review'][1]
a_text

In [ ]:
a_text = clean_review(a_text)
a_text

In [ ]:
a_text = word_tokenize(a_text)
print(a_text)

In [ ]:
a_text = lemmatize(a_text)
print(a_text)

### Algumas metricas

In [ ]:
df_train['review_lenght'] = df_train['review'].apply(len)
median = df_train['review_lenght'].median()
mean   = df_train['review_lenght'].mean()
mode   = df_train['review_lenght'].mode()[0]
df_train.head(1)

In [ ]:
fig, ax = plt.subplots()

sns.distplot(df_train['review_lenght'], bins=df_train['review_lenght'].max(),
            hist_kws={"alpha": 0.9, "color": "blue"}, ax=ax,
            kde_kws={"color": "black", 'linewidth': 3})

ax.set_xlim(left=0, right=np.percentile(df_train['review_lenght'], 95))
ax.set_xlabel('Words in review')

ymax = 0.014
plt.ylim(0, ymax)
ax.plot([mode, mode], [0, ymax], '--', label=f'mode = {mode:.2f}', linewidth=4)
ax.plot([mean, mean], [0, ymax], '--', label=f'mean = {mean:.2f}', linewidth=4)
ax.plot([median, median], [0, ymax], '--',
        label=f'median = {median:.2f}', linewidth=4)

ax.set_title('Words per review distribution', fontsize=20)

plt.legend()
plt.show()

In [ ]:
t0 = time.time()

bigrams = Phrases(sentences=df_train['review'])

print(time_spent(t0)) # 30s

In [ ]:
t0 = time.time() 

trigrams = Phrases(sentences=bigrams[df_train['review']])

print(time_spent(t0)) # 1min 22s

In [ ]:
t0 = time.time() 

embedding_vector_size = 256
trigrams_model = Word2Vec(
    sentences = trigrams[bigrams[df_train['review']]],
    size = embedding_vector_size,
    min_count=3, window=5, workers=4)

print(time_spent(t0)) # 12min 40s

In [ ]:
print("Vocabulary size:", len(trigrams_model.wv.vocab))

In [ ]:
trigrams_model.wv.most_similar('galaxy')

In [ ]:
trigrams_model.wv.doesnt_match(['galaxy', 'starship', 'planet', 'dog'])

In [ ]:
# Tools for building a model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
%%time
def vectorize_data(data, vocab: dict) -> list:
    print('Vectorize sentences...', end='\r')
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    print('Vectorize sentences... (done)')
    return vectorized

print('Convert sentences to sentences with ngrams...', end='\r')
X_data = trigrams[bigrams[df_train['review']]]
print('Convert sentences to sentences with ngrams... (done)')

input_length = 150
X_pad = pad_sequences(
    sequences=vectorize_data(X_data, vocab=trigrams_model.wv.vocab),
    maxlen=input_length,
    padding='post')
print('Transform sentences to sequences... (done)')

# 22min

In [ ]:
# Word2Vec: each word as a array of 150
X_pad[15000]

In [ ]:
Y_train_data = df_train.sentiment.values

X_train, X_test, y_train, y_test = train_test_split(
    X_pad,
    Y_train_data,
    test_size=0.05,
    shuffle=True,
    random_state=42)

In [ ]:
%%time
# Tools for building a model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

def build_model(embedding_matrix: np.ndarray, input_length: int):
    model = Sequential()
    model.add(Embedding(
        input_dim = embedding_matrix.shape[0],
        output_dim = embedding_matrix.shape[1], 
        input_length = input_length,
        weights = [embedding_matrix],
        trainable=False))
    model.add(Bidirectional(LSTM(128, recurrent_dropout=0.1)))
    model.add(Dropout(0.25))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

model = build_model(
    embedding_matrix=trigrams_model.wv.vectors,
    input_length=input_length)

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy'])

history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    batch_size=150,
    epochs=15)

# Take 2 hours

In [ ]:
plot_nn_loss_acc(history)

In [ ]:
from sklearn.metrics import classification_report

y_train_pred = model.predict_classes(X_train)
y_test_pred = model.predict_classes(X_test)

class_report(y_train, y_train_pred, 'Test Strategie 4: Neural Net LSTM in Train')
class_report(y_test, y_test_pred, 'Test Strategie 4: Neural Net LSTM in Test')

In [ ]:
df_to_submit = pd.concat([df_sub_test])
df_to_submit['review'] = df_to_submit['review'].apply(preprocess)

In [ ]:
%%time
# 8 min
print('Convert sentences to sentences with ngrams...', end='\r')
X_submit = trigrams[bigrams[df_to_submit['review']]]
print('Convert sentences to sentences with ngrams... (done)')

X_sub = pad_sequences(
    sequences=vectorize_data(X_submit, vocab=trigrams_model.wv.vocab),
    maxlen=input_length,
    padding='post')
print('Transform sentences to sequences... (done)')

In [ ]:
%%time
Y_sub_pred = model.predict_classes(X_sub)
df_to_submit['sentiment'] = model.predict_classes(X_sub)
df_to_submit.to_csv('submission_dtrategie4_nn_moretime.csv', index=False, columns=['id','sentiment']) # Kaggle: 0.98068

In [ ]:
%%time

from keras.models import model_from_json

################################################

# SAVE

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

################################################

# LOAD

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

################################################

y_train_pred = loaded_model.predict_classes(X_train)
y_test_pred = loaded_model.predict_classes(X_test)

from sklearn.metrics import classification_report

target_names = ['negatives', 'positives']

print(classification_report(y_train, y_train_pred, target_names=target_names))
print(classification_report(y_test, y_test_pred, target_names=target_names))